In [3]:
import pandas as pd
import numpy as np
import xlwt
import xlrd
from tkinter import filedialog
import tkinter as tk
import customtkinter as ctk
from customtkinter import filedialog
import funciones
import openpyxl
import dbfread
import glob
from pandastable import Table, TableModel # Importar clases de pandastable

### Subir reporte danilo

In [4]:
archivo = None
while True:
    archivo=funciones.seleccionar_archivo()
    print(archivo)
    if archivo:
        ruta_archivo = archivo.name
        try:
            
            #dbf = dbfread.DBF(ruta_archivo)
            monotributo = pd.read_excel(ruta_archivo, sheet_name=0)
            print(monotributo)
            break
        except UnicodeDecodeError:
            print("Error de decodificación de caracteres. Intenta con otra codificación o archivo.")
            continue
    else:
        print("Archivo incorrecto. Por favor, vuelva a cargarlo.")

<_io.TextIOWrapper name='C:/Users/zickd/Downloads/transferncias AFIP reporte de errores en importacion Gecros desde el 15-09-23 al 30-09-23.xlsx' mode='r' encoding='cp1252'>
           cuit  cuil                                    descripcion  \
0   20406560814   NaN  401: Contrib. Patr. sin Empl. para distribuir   
1   20406560814   NaN  401: Contrib. Patr. sin Empl. para distribuir   
2   20231981188   NaN  401: Contrib. Patr. sin Empl. para distribuir   
3   20231981188   NaN  401: Contrib. Patr. sin Empl. para distribuir   
4   20231981188   NaN  401: Contrib. Patr. sin Empl. para distribuir   
..          ...   ...                                            ...   
92  27181363105   NaN  401: Contrib. Patr. sin Empl. para distribuir   
93  27453149183   NaN  401: Contrib. Patr. sin Empl. para distribuir   
94  27453149183   NaN  401: Contrib. Patr. sin Empl. para distribuir   
95  27453149183   NaN  401: Contrib. Patr. sin Empl. para distribuir   
96  27453149183   NaN  401: Contri

In [5]:
monotributo.count()

cuit           97
cuil            0
descripcion    97
obrasocial      0
periodo        97
sueldo         97
aporte         97
contri         97
otrosconc      97
fecha          97
dtype: int64

###  Importar padron total gecros monotrbutistas

In [6]:
#cargar primero el padron de sidefi
archivo = None
while True:
    archivo = funciones.seleccionar_archivo()
    print(archivo)
    if archivo:
        ruta_archivo = archivo.name
        try:
            padron = pd.read_excel(ruta_archivo)
            #padron = pd.read_csv(archivo, sep=";", encoding="ISO-8859-1")
            print(padron)
            break
        except UnicodeDecodeError:
            print("Error de decodificación de caracteres. Intenta con otra codificación o archivo.")
            continue
    else:
        print("Archivo incorrecto. Por favor, vuelva a cargarlo.")

<_io.TextIOWrapper name='C:/Users/zickd/Downloads/PADRON_GECROS.xlsx' mode='r' encoding='cp1252'>
       OS_NOMBRE          CUIL SEXO      NOMPLAN PARENTESCO  \
0      EVP (TIS)           NaN    F  Desregulado     HIJO/A   
1      EVP (TIS)           NaN    F  Desregulado     HIJO/A   
2      EVP (TIS)           NaN    F  Desregulado    TITULAR   
3      EVP (TIS)           NaN    F  Desregulado    TITULAR   
4      EVP (TIS)           NaN    F  Desregulado    TITULAR   
...          ...           ...  ...          ...        ...   
66942    VISITAR  2.756850e+10    F    Afiliados     HIJO/A   
66943    VISITAR  2.758200e+10    F    Afiliados     HIJO/A   
66944    VISITAR  2.758853e+10    F    Afiliados     HIJO/A   
66945    VISITAR  2.793704e+10    F    Afiliados    TITULAR   
66946    VISITAR  2.795855e+10    F    Afiliados    TITULAR   

                APELLIDO              NOMBRES                  TIPOBEN_NO  
0                  PEREZ       ORIANA GISSELE    07-MONOT. EFECTOR SO

In [7]:
padron['CUIL'] = pd.to_numeric(padron['CUIL'], errors='coerce').fillna(0).astype('Int64')
padron['CUIL'].head()

0    0
1    0
2    0
3    0
4    0
Name: CUIL, dtype: Int64

#### Cruce de archivos! 

In [8]:
liquidaciones_monot = pd.merge(monotributo, padron, left_on= 'cuit', right_on='CUIL', how='inner', indicator=True)
liquidaciones_monot["Razón Social"]=" "
liquidaciones_monot['periodo'] = liquidaciones_monot['periodo'].astype(str).apply(lambda x: "20" + x)
liquidaciones_monot['periodo'] = liquidaciones_monot['periodo'].astype(int)
liquidaciones_monot.columns

Index(['cuit', 'cuil', 'descripcion', 'obrasocial', 'periodo', 'sueldo',
       'aporte', 'contri', 'otrosconc', 'fecha', 'OS_NOMBRE', 'CUIL', 'SEXO',
       'NOMPLAN', 'PARENTESCO', 'APELLIDO', 'NOMBRES', 'TIPOBEN_NO', '_merge',
       'Razón Social'],
      dtype='object')

In [9]:
print(liquidaciones_monot["periodo"])

0     202208
1     202303
2     202204
3     202205
4     202206
5     202207
6     202208
7     202209
8     202210
9     202211
10    202212
11    202301
12    202302
13    202303
14    202304
15    202306
16    202307
17    202303
18    202306
19    202308
20    202306
21    202307
22    202308
23    201912
Name: periodo, dtype: int32


In [10]:
aportes_a_importar = liquidaciones_monot[["cuit",
"Razón Social",
"periodo",
"fecha",
"CUIL",
"APELLIDO",
"NOMBRES",
"sueldo",
"aporte",
"contri",
"otrosconc"
]]

In [11]:
path = funciones.seleccionar_directorio()
liquidaciones_monot.to_excel(path + '\cruce_desde el 15-09-23 al 30-09-23_raw.xlsx', index=False)
aportes_a_importar.to_excel(path + '\cruce_desde el 15-09-23 al 30-09-23.xls', index = False, engine='xlwt', header = False)

C:\Users\zickd\AppData\Local\Temp\ipykernel_13652\383208089.py:3: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  aportes_a_importar.to_excel(path + '\cruce_desde el 15-09-23 al 30-09-23.xls', index = False, engine='xlwt', header = False)


In [12]:
aportes_a_importar.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24 entries, 0 to 23
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   cuit          24 non-null     int64         
 1   Razón Social  24 non-null     object        
 2   periodo       24 non-null     int32         
 3   fecha         24 non-null     datetime64[ns]
 4   CUIL          24 non-null     Int64         
 5   APELLIDO      24 non-null     object        
 6   NOMBRES       24 non-null     object        
 7   sueldo        24 non-null     int64         
 8   aporte        24 non-null     int64         
 9   contri        24 non-null     float64       
 10  otrosconc     24 non-null     int64         
dtypes: Int64(1), datetime64[ns](1), float64(1), int32(1), int64(4), object(3)
memory usage: 2.2+ KB
